In [38]:
import psycopg2
import pandas as pd

In [39]:
DBNAME = "test"
HOST = "35.202.86.236"
USER = "alex"
PASSWORD = "alex"
PORT = "5432"

conn = psycopg2.connect(dbname=DBNAME, user=USER, password=PASSWORD, host=HOST, port=PORT)


In [40]:
cursor = conn.cursor()

In [41]:
USER = '9b8e372a-b5d4-4771-8741-511bae5d9193'

cursor.execute("""
SELECT
    user_uuid,
    log_id,
    protein,
    carbohydrates,
    fats,
    calories,
    servings,
    log_date,
    LOWER(TRIM(food_name)) as food_name,
    serving_size,
    group_id,
    CASE
        WHEN EXTRACT(HOUR FROM log_date) >= 0 AND EXTRACT(HOUR FROM log_date) < 6 THEN 'Night'
        WHEN EXTRACT(HOUR FROM log_date) >= 6 AND EXTRACT(HOUR FROM log_date) < 12 THEN 'Morning'
        WHEN EXTRACT(HOUR FROM log_date) >= 12 AND EXTRACT(HOUR FROM log_date) < 18 THEN 'Afternoon'
        ELSE 'Evening'
    END AS time_of_day
FROM
    food_log
WHERE
    user_uuid = '{}'
GROUP BY 
    user_uuid,
    log_id,
    protein,
    carbohydrates,
    fats,
    calories,
    servings,
    log_date,
    food_name,
    serving_size,
    group_id,
    time_of_day

""".format(USER))

records = cursor.fetchall()

# Create a DataFrame from the fetched records
df = pd.DataFrame(records, columns=[
    'user_uuid',
    'log_id',
    'protein',
    'carbohydrates',
    'fats',
    'calories',
    'servings',
    'log_date',
    # 'food_name_clean',
    'food_name',
    # 'image_path',
    'serving_size',
    'group_id',
    'time_of_day'

])


cursor.close()
conn.close()

# # Save to file
df.to_csv('./data/user.csv', index=False)
df

,user_uuid,log_id,protein,carbohydrates,fats,calories,servings,log_date,food_name,serving_size,group_id,time_of_day
0,9b8e372a-b5d4-4771-8741-511bae5d9193,500,0,22,0,90,1,2023-07-20 02:49:50,Sweet Tea,8 fl oz,202307200249509b8e372a-b5d4-4771-8741-511bae5d...,Night
1,9b8e372a-b5d4-4771-8741-511bae5d9193,260,28,41,18,440,1,2023-07-01 01:11:00,Chick-fil-A sandwich,1 sandwich,202307010111009b8e372a-b5d4-4771-8741-511bae5d...,Night
2,9b8e372a-b5d4-4771-8741-511bae5d9193,262,28,40,18,440,1,2023-07-01 01:11:37,Chick-fil-A sandwich,1 sandwich,202307010111379b8e372a-b5d4-4771-8741-511bae5d...,Night
3,9b8e372a-b5d4-4771-8741-511bae5d9193,575,28,9,13,260,1,2023-09-01 02:38:02,Chick-fil-A 8 count nugget,8 pieces,202309010238029b8e372a-b5d4-4771-8741-511bae5d...,Night
4,9b8e372a-b5d4-4771-8741-511bae5d9193,670,6,58,20,410,1,2023-09-19 00:19:42,chocolate chip muffin,1 muffin,202309190019429b8e372a-b5d4-4771-8741-511bae5d...,Night
...,...,...,...,...,...,...,...,...,...,...,...,...
359,9b8e372a-b5d4-4771-8741-511bae5d9193,2807,24,3,2,120,1,2024-04-28 18:55:26,optimum nutrition scoop of whey,scoop/g,202404281855269b8e372a-b5d4-4771-8741-511bae5d...,Evening
360,9b8e372a-b5d4-4771-8741-511bae5d9193,2808,0,0,0,148,2,2024-04-28 18:55:26,cups of milk,cup,202404281855269b8e372a-b5d4-4771-8741-511bae5d...,Evening
361,9b8e372a-b5d4-4771-8741-511bae5d9193,2842,4,30,2,151,2,2024-04-30 00:47:41,bags of oatmeal,varies,202404300047419b8e372a-b5d4-4771-8741-511bae5d...,Night
362,9b8e372a-b5d4-4771-8741-511bae5d9193,2877,80,21,10,480,1,2024-04-30 19:01:00,optimum nutrition shake with 2 cups of milk,2 scoops + 2 cups milk,202404301901009b8e372a-b5d4-4771-8741-511bae5d...,Evening


In [42]:
user = pd.read_csv('./data/user.csv')

userData = user[['food_name', 'log_date', 'time_of_day']].copy()  # Make a copy of the DataFrame

# Extract weekday from log_date and assign it to a new column 'weekday' using .loc
userData.loc[:, 'weekday'] = pd.to_datetime(userData['log_date']).dt.day_name()

# userData.loc[:, 'time'] = pd.to_datetime(userData['log_date']).dt.time

userData.drop(['log_date'], axis=1, inplace=True)  # Drop the 'log_date' column

userData['food_name'] = userData['food_name'].str.lower()  # Capitalize the food names

userData.to_csv('./data/userData.csv', index=False)  # Save to file

userData


,food_name,time_of_day,weekday
0,sweet tea,Night,Thursday
1,chick-fil-a sandwich,Night,Saturday
2,chick-fil-a sandwich,Night,Saturday
3,chick-fil-a 8 count nugget,Night,Friday
4,chocolate chip muffin,Night,Tuesday
...,...,...,...
359,optimum nutrition scoop of whey,Evening,Sunday
360,cups of milk,Evening,Sunday
361,bags of oatmeal,Night,Tuesday
362,optimum nutrition shake with 2 cups of milk,Evening,Tuesday
